<a href="https://colab.research.google.com/github/forexms78/AI-05-/blob/main/%EB%AF%B8%EC%85%989_7%ED%8C%80_%EB%B0%95%EB%B3%91%ED%98%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 모델을 활용하여 FashionMNIST 데이터셋을 조건부로 생성하는 작업을 수행
#cGAN (Conditional GAN) 모델

In [1]:
#@title 라이브러리

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import torchvision.utils as vutils
import matplotlib.pyplot as plt
import numpy as np
import random

In [2]:
#@title Device 설정

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cuda


In [4]:
# @title 드라이브 연결

from google.colab import drive
drive.mount('/content/drive')
g_path = '/content/drive/MyDrive/Colab Notebooks/data'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
#@title 일정한 결과값 확인을 위한 seed 설정
seed = 42
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
random.seed(seed)
np.random.seed(seed)

In [6]:
#@title 하이퍼파라미터 및 환경 설정

image_size = 28
num_classes = 10
latent_dim = 100
batch_size = 64
num_epochs = 200
lr = 0.0002
beta1 = 0.5
beta2 = 0.999

In [8]:
#@title Transform 설정

# 학습용 데이터 증강 transform
train_transforms = transforms.Compose([
    transforms.RandomHorizontalFlip(p=0.2),          # 20% 확률로 좌우 반전
    transforms.RandomRotation(10),                     # ±10도 범위 내에서 랜덤 회전
    transforms.RandomAffine(degrees=0, translate=(0.1, 0.1)),  # 최대 10% 이동
    transforms.RandomResizedCrop(28, scale=(0.8, 1.0), ratio=(0.9, 1.1)),  # 랜덤 크롭 후 28x28로 리사이즈
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

# 평가용 transform (증강 없이 정규화만)
test_transforms = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])


In [9]:
#@title 데이터 셋, 데이터 로더
train_dataset = torchvision.datasets.FashionMNIST(root='./data', train=True, download=True, transform=train_transforms)
test_dataset  = torchvision.datasets.FashionMNIST(root='./data', train=False, download=True, transform=test_transforms)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader  = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


100%|██████████| 26.4M/26.4M [00:00<00:00, 113MB/s]
100%|██████████| 29.5k/29.5k [00:00<00:00, 3.95MB/s]
100%|██████████| 4.42M/4.42M [00:00<00:00, 53.5MB/s]
100%|██████████| 5.15k/5.15k [00:00<00:00, 20.8MB/s]


In [10]:
#@title 클래스 이름 설정 (FashionMNIST 클래스)

idx_to_class = {i: class_name for i, class_name in enumerate(train_dataset.classes)}
print("FashionMNIST classes:", idx_to_class)

FashionMNIST classes: {0: 'T-shirt/top', 1: 'Trouser', 2: 'Pullover', 3: 'Dress', 4: 'Coat', 5: 'Sandal', 6: 'Shirt', 7: 'Sneaker', 8: 'Bag', 9: 'Ankle boot'}
